In [15]:
import pandas as pd
samples_path = '../logs/samples.csv'
samples = pd.read_csv(samples_path, index_col=0)
samples = [(s[0:25].tolist(), s[25:].tolist()) for s in samples.values][0:2]

In [20]:
nnet_path = '../network/models/v3.2.0/model.nnet'
test_local_robustness(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, verbose=2)

INFO:robustness:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:robustness:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:robustness:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:robustness:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:robustness:searching 9 coarse epsilons b/t 0.01 and 0.09
INFO:robustness:s0 coarse epsilon bounds: (0.02, 0.03) (2187ms)
INFO:robustness:searching 1000 epsilons b/t 0.02 and 0.02999
INFO:robustness:s0 epsilon: 0.02147 (780ms)
INFO:robustness:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:robustness:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:robustness:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:robustness:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:robustness:s1 coarse epsilon bounds: (0.001, 0.002) (1908ms)
INFO:robustness:searching 100 epsilons b/t 0.001 and 0.00199
INFO:robustness:s1 epsilon: 0.00124 (609ms)
INFO:robustness:local robustness: (-0.00124, 0.00124) (5486ms)


((-0.00124, 0.00124), [(0.02147, 0.02147), (0.00124, 0.00124)])

In [2]:
from robustness import test_local_robustness

nnet_path = '../network/models/v3.2.0/model.nnet'
sym_results = test_local_robustness(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, save_results=True, save_samples=True, verbose=2, outdir='../logs/test/robustness/symmetric')
asym_results = test_local_robustness(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, asym=True, save_results=True, save_samples=True, verbose=2, outdir='../logs/test/robustness/asymmetric')

print('Symmetric results: ', sym_results)
print('Asymmetric results: ', asym_results)

INFO:robustness:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:robustness:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:robustness:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:robustness:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:robustness:searching 9 coarse epsilons b/t 0.01 and 0.09
INFO:robustness:s0 coarse epsilon bounds: (0.01, 0.02) (2242ms)
INFO:robustness:searching 1000 epsilons b/t 0.01 and 0.01999
INFO:robustness:s0 epsilon: 0.01606 (1115ms)
INFO:robustness:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:robustness:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:robustness:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:robustness:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:robustness:s1 coarse epsilon bounds: (0.005, 0.006) (1847ms)
INFO:robustness:searching 100 epsilons b/t 0.005 and 0.00599
INFO:robustness:s1 epsilon: 0.00533 (535ms)
INFO:robustness:local robustness: (-0.00533, 0.00533) (5743ms)
INFO:robustness:wr

In [3]:
from robustness import check_local_robustness

nnet_path = '../network/models/v3.2.0/model.nnet'
sym_check_results = check_local_robustness(nnet_path, samples, sym_results)
asym_check_results = check_local_robustness(nnet_path, samples, asym_results, asym=True)

print('Symmetric check results:', sym_check_results)
print('Asymmetric check results:', asym_check_results)

INFO:robustness:s1 ok (-0.00533, 0.00533)
INFO:robustness:s1 ok (-0.0072, 0.00606)
Symmetric check results: {'s0': None, 's1': None}
Asymmetric check results: {'s0': (None, None), 's1': (None, None)}


In [3]:
from clustering import LGKUtils

lgkmc = LGKUtils.load('../artifacts/lgkm.pkl')
regions = lgkmc.get_regions(sort=True)
r0 = regions[0]